# Exploratory Data Analysis of Epicurious Scrape in a JSON file

This is an idealized workflow for Aaron Chen in looking at data science problems. It likely isn't the best path, nor has he rigidly applied or stuck to this ideal, but he wishes that he worked this way more frequently.

## Purpose: Work through some exploratory data analysis of the Epicurious scrape on stream. Try to write some functions to help process the data.

### Author: Aaron Chen


---

### If needed, run shell commands here

In [1]:
# !python -m spacy download en_core_web_sm

---

## External Resources

List out references or documentation that has helped you with this notebook

### Code
Regex Checker: https://regex101.com/

#### Scikit-learn
1. https://scikit-learn.org/stable/modules/decomposition.html#latent-dirichlet-allocation-lda
2. 

### Data

For this notebook, the data is stored in the repo base folder/data/raw

### Process

Are there steps or tutorials you are following? Those are things I try to list in Process

___

## Import necessary libraries

In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.base import TransformerMixin
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lemmatizer import Lemmatizer
from tqdm import tqdm
from typing import Any

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


---

## Define helper functions

My workflow is to try things with code cells, then when the code cells get messy and repetitive, to convert into helper functions that can be called.

When the helper functions are getting used a lot, it is usually better to convert them to scripts or classes that can be called/instantiated

In [2]:
def custom_lemmatizer(ingredients: list) -> Any: # spacy nlp.Doc
    """This takes in a string representing the recipe and an NLP model and lemmatize with the NER. 
    
    Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    Remove punctuation

    Args:
        ingredients: string
        nlp_mod: spacy model (try built in first, by default called nlp)
    
    Returns:
        nlp.Doc
    """
    lemmas = [token.lemma_ for token in ingredients if (token.is_alpha and token.pos_ not in ["PRON", "VERB"] and len(token.lemma_) > 1)]
    return lemmas
    # return doc

In [3]:
def custom_preprocessor(recipe_ingreds: str) -> list:
    """This function replaces the default sklearn CountVectorizer preprocessor to use spaCy. sklearn CountVectorizer's preprocessor only performs accent removal and lowercasing.

    Args:
        A string to tokenize from a recipe representing the ingredients used in the recipe

    Returns:
        A list of strings that have been de-accented and lowercased to be used in tokenization
    """
    preprocessed = [token for token in nlp(recipe_ingreds)]

    return preprocessed

In [4]:
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

### Import local script

I started grouping this in with importing libraries, but putting them at the bottom of the list

In [5]:
import project_path

import src.dataframe_preprocessor as dfpp

pyLDAvis.enable_notebook()

---

## Define global variables 
### Remember to refactor these out, not ideal

In [6]:
data_path = "../../data/recipes-en-201706/epicurious-recipes_m2.json"
food_stopwords_path = "../../food_stopwords.csv"

---

## Running Commentary

1. I used numbered lists to keep track of things I noticed

### To Do

1. Try to determine consistency of nested data structures
   1. Is the photoData or number of things inside photoData the same from record to record
   2. What about for tag?

Data wasn't fully consistent but logic in helper function helped handle nulls

2. How to handle nulls?
   1. Author      Filled in with "Missing Author"
   2. Tag         Filled in with "Missing Cuisine"
3. ~~Convert pubDate to actual timestamp~~  
4. ~~Convert ScrapeDate to actual timestamp~~
   1. This was ignored as the datestamp was not useful (generally within minutes of the origin of UNIX time)
   
**5. Append new columns for relevant nested structures and unfold them**

6. Determine actual types of `ingredients` and `prepSteps`
7. Continue working through test example of single recipe to feed into spaCy and then sklearn.feature_extraction.text stack
8. Will need to remove numbers, punctuation

---

## Importing and viewing the data as a dataframe

In [7]:
repo = pd.read_json(path_or_buf=data_path) # type:ignore
pd.read_json(data_path, typ='frame') # type:ignore

dfpp.preprocess_dataframe(df=repo) # type:ignore
print(repo.shape)
repo.head(10) # type:ignore

recipe_megalist = [ingred for recipe in repo['ingredients'].tolist() for ingred in recipe]


nlp = spacy.load("en_core_web_sm")

# this is a redeem for variable naming mixed with a free pun-ish me daddy, flushtrated will be the list of all stopword to exclude so named because we're throwing these words down the drain

flushtrated = {x for x in pd.read_csv(food_stopwords_path)}
flushtrated = flushtrated.union(STOP_WORDS)
flushtrated_list = list(flushtrated)

(34656, 14)


In [8]:
cv = CountVectorizer(strip_accents='unicode', lowercase=True, preprocessor=custom_preprocessor, tokenizer=custom_lemmatizer, stop_words=flushtrated_list, ngram_range=(1,4))
repo_transformed = cv.fit_transform(tqdm(recipe_megalist))
cv.get_feature_names_out().shape

  0%|          | 0/341271 [00:00<?, ?it/s]/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Frank', 'alternative', 'american', 'annie', 'asian', 'balance', 'band', 'barrel', 'bay', 'bayou', 'beam', 'beard', 'bell', 'betty', 'bird', 'blast', 'bob', 'bone', 'breyers', 'calore', 'carb', 'card', 'chachere', 'change', 'circle', 'coffee', 'coil', 'country', 'cow', 'crack', 'cracker', 'crocker', 'crystal', 'dean', 'degree', 'deluxe', 'direction', 'duncan', 'earth', 'eggland', 'ener', 'envelope', 'eye', 'fantastic', 'far', 'fat', 'feather', 'flake', 'foot', 'fourth', 'frank', 'french', 'fry', 'fusion', 'genoa', 'genovese', 'germain', 'giada', 'gold', 'golden', 'granule', 'greek', 'hamburger', 'helper', 'herbe', 'hines', 'hodgson', 'hunt', 'instruction', 'interval', 'italianstyle', 'jim', 'jimmy', 'kellogg', 'lagrille'

(79401,)

We can try to filter out the adjectives in the lemmatization step, because spaCy allows filtering based on Parts of Speech. But this might exclude them from the ngrams. Let's try augmenting stopwords and excluding colors that way.

In [9]:
lda_20 = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
lda_20_repo_transformed = lda_20.fit_transform(repo_transformed)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    3.0s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    3.1s remaining:   33.6s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    3.1s remaining:   22.0s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    3.3s remaining:   16.3s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    3.3s remaining:   12.6s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    3.4s remaining:   10.1s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    3.4s remaining:    8.3s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    3.4s remaining:    6.9s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    3.5s remaining:    5.8s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    3.5s remaining:    4.9s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    3.5s remaining:    4.2s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

In [10]:
pyLDAvis.sklearn.prepare(lda_20, repo_transformed, cv)

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.9s remaining:    9.8s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    0.9s remaining:    6.3s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    0.9s remaining:    4.5s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    0.9s remaining:    3.4s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.9s remaining:    2.7s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    0.9s remaining:    2.2s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    0.9s remaining:    1.8s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    0.9s remaining:    1.5s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    0.9s remaining:    1.3s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.9s remaining:    1.1s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5      0.057976 -0.220075       1        1  7.404274
3     -0.315038 -0.035260       2        1  6.978373
17    -0.344727  0.077418       3        1  6.334855
4      0.022988  0.075638       4        1  6.283618
19     0.015151 -0.223425       5        1  5.932245
9      0.032503  0.131887       6        1  5.330042
1      0.001952 -0.202480       7        1  5.319318
13     0.010691 -0.029209       8        1  4.893098
8     -0.015862 -0.115547       9        1  4.890017
14     0.041128  0.172688      10        1  4.875664
7      0.056815  0.039933      11        1  4.789695
10     0.074877  0.059133      12        1  4.499205
12    -0.035030  0.000239      13        1  4.470400
16     0.073392 -0.032828      14        1  4.380264
2      0.061838  0.008674      15        1  4.276843
18     0.063887  0.019079      16        1  4.242786
0      0.041084  0.026884      17        1  4.057456
15     0.086683  0.046194      18        1  4.048753
6      0.023584  0.130641      19        1  3.513499
11     0.046111  0.070417      20        1  3.479594, topic_info=            Term          Freq         Total Category  logprob  loglift
65558       salt  17477.000000  17477.000000  Default  30.0000  30.0000
71207      sugar  13419.000000  13419.000000  Default  29.0000  29.0000
54591        oil  16991.000000  16991.000000  Default  28.0000  28.0000
55095      olive  11694.000000  11694.000000  Default  27.0000  27.0000
55219  olive oil  10611.000000  10611.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
22766       beef    262.846721   1454.319053  Topic20  -4.7623   1.6475
20740       baby    204.341893   1314.883420  Topic20  -5.0140   1.4965
43339     ginger    192.134854   2346.833553  Topic20  -5.0756   0.8556
29359     center    135.661901    345.485901  Topic20  -5.4237   2.4235
24900       bone    133.290945    769.148958  Topic20  -5.4413   1.6055

[908 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
524       12  0.979507     Arborio
838       17  0.985175       Baker
1679      19  0.097797        Bosc
1679      20  0.899731        Bosc
1690      20  0.995101   Bosc pear
...      ...       ...         ...
79153      5  0.266331        zest
79153     11  0.164918        zest
79153     18  0.568107        zest
79176     11  0.997719  zest lemon
79256     14  0.998652    zucchini

[1204 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 18, 5, 20, 10, 2, 14, 9, 15, 8, 11, 13, 17, 3, 19, 1, 16, 7, 12])

## Manual Topic Labeling Based on LDA
1. aliums, alium prep, chocolate and rosemary
2. oil, cheese, game meats
3. peppers and parsley

Based on these three topics, I think it is better to train a new model, since these models don't seem to carry much information

In [13]:
cv_auto_stopwords_085 = CountVectorizer(strip_accents='unicode', lowercase=True, preprocessor=custom_preprocessor, tokenizer=custom_lemmatizer, stop_words=None, ngram_range=(1,4), max_df=0.85)

repo_transformed_auto_stopwords_085 = cv_auto_stopwords_085.fit_transform(tqdm(recipe_megalist))
cv_auto_stopwords_085.get_feature_names_out().shape
lda_20_auto_stopwords_085 = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
lda_20_repo_transformed_auto_stopwords_085 = lda_20_auto_stopwords_085.fit_transform(repo_transformed_auto_stopwords_085)
pyLDAvis.sklearn.prepare(lda_20_auto_stopwords_085, repo_transformed_auto_stopwords_085, cv_auto_stopwords_085)

100%|██████████| 341271/341271 [15:22<00:00, 369.77it/s]


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:   10.9s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:   11.0s remaining:  2.0min
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:   11.3s remaining:  1.3min
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:   11.5s remaining:   57.3s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:   11.7s remaining:   44.3s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:   11.8s remaining:   35.5s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:   12.2s remaining:   29.7s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:   12.4s remaining:   24.8s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:   12.5s remaining:   20.9s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:   12.7s remaining:   17.8s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:   12.9s remaining:   15.2s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    2.8s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    3.0s remaining:   33.4s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    3.0s remaining:   21.3s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    3.1s remaining:   15.3s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    3.1s remaining:   11.8s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    3.4s remaining:   10.1s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    3.5s remaining:    8.4s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    3.5s remaining:    7.0s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    3.5s remaining:    5.8s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    3.5s remaining:    4.9s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    3.6s remaining:    4.3s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.278588  0.160599       1        1  8.155682
15    -0.009181 -0.156219       2        1  7.248863
12    -0.165521  0.106973       3        1  6.474569
17    -0.129976 -0.066788       4        1  5.847223
18     0.107470 -0.053242       5        1  5.441509
9     -0.190586  0.026811       6        1  5.406021
19     0.029001 -0.190125       7        1  5.269574
6      0.185226  0.166514       8        1  5.226811
4      0.088543 -0.061766       9        1  5.141434
10    -0.036628 -0.164227      10        1  4.938742
8      0.104500  0.045119      11        1  4.424080
5     -0.015968 -0.079803      12        1  4.384623
13     0.062989  0.002264      13        1  4.288278
14     0.152703  0.138450      14        1  4.211380
2     -0.013727 -0.009903      15        1  4.144772
7      0.093250  0.111337      16        1  4.120203
3     -0.006058 -0.061775      17        1  3.946463
1      0.039292 -0.008339      18        1  3.922857
11    -0.005947  0.096131      19        1  3.886079
16    -0.010794 -0.002011      20        1  3.520837, topic_info=              Term          Freq         Total Category  logprob  loglift
455821    teaspoon  48583.000000  48583.000000  Default  30.0000  30.0000
101289        chop  29353.000000  29353.000000  Default  29.0000  29.0000
393827        salt  17956.000000  17956.000000  Default  28.0000  28.0000
125220         cup  91329.000000  91329.000000  Default  27.0000  27.0000
184920       fresh  26525.000000  26525.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
100393   chocolate    432.001560   2602.724159  Topic20  -5.8697   1.5506
403111     shallot    423.919339   2212.934175  Topic20  -5.8886   1.6939
180608        food    395.780495   1135.640740  Topic20  -5.9573   2.2924
252306  lengthwise    408.052817   2640.419019  Topic20  -5.9268   1.4792
313710   ounce can    405.369017   2279.019969  Topic20  -5.9334   1.6198

[1156 rows x 6 columns], token_table=        Topic      Freq             Term
term                                    
7574       13  0.593582            Dijon
7574       18  0.405664            Dijon
7581       13  0.557614    Dijon mustard
7581       18  0.441342    Dijon mustard
11357       6  0.997991            Grand
...       ...       ...              ...
508640     13  0.998383            yeast
508934     11  0.997765  yellow cornmeal
509591      7  0.941378           yogurt
509591     17  0.057694           yogurt
509868     15  0.999418             yolk

[2546 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 16, 13, 18, 19, 10, 20, 7, 5, 11, 9, 6, 14, 15, 3, 8, 4, 2, 12, 17])

This LDA is probably less useful

1. Has a lot of prep and units of measurement
2. 

In [12]:
additional_to_exclude = {'red', 'green', 'black', 'yellow', 'white', 'inch', 'mince', 'chop', 'fry', 'trim', 'flat', 'beat', 'brown', 'golden', 'balsamic', 'halve', 'blue', 'divide', 'trim', 'unbleache', 'granulate'}
flushtrated_augment = flushtrated.union(additional_to_exclude)
flushtrated_augment = list(flushtrated_augment)

cv_stopwords_aug = CountVectorizer(strip_accents='unicode', lowercase=True, preprocessor=custom_preprocessor, tokenizer=custom_lemmatizer, stop_words=flushtrated_augment, ngram_range=(1,4))

repo_transformed_stopwords_aug = cv_stopwords_aug.fit_transform(tqdm(recipe_megalist))
cv_stopwords_aug.get_feature_names_out().shape
lda_20_stopwords_aug = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
lda_20_repo_transformed_aug = lda_20_stopwords_aug.fit_transform(repo_transformed_stopwords_aug)
pyLDAvis.sklearn.prepare(lda_20_stopwords_aug, repo_transformed_stopwords_aug, cv_stopwords_aug)

  0%|          | 0/341271 [00:00<?, ?it/s]/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Frank', 'alternative', 'american', 'annie', 'asian', 'balance', 'band', 'barrel', 'bay', 'bayou', 'beam', 'beard', 'bell', 'betty', 'bird', 'blast', 'bob', 'bone', 'breyers', 'calore', 'carb', 'card', 'chachere', 'change', 'circle', 'coffee', 'coil', 'country', 'cow', 'crack', 'cracker', 'crocker', 'crystal', 'dean', 'degree', 'deluxe', 'direction', 'duncan', 'earth', 'eggland', 'ener', 'envelope', 'eye', 'fantastic', 'far', 'fat', 'feather', 'flake', 'foot', 'fourth', 'frank', 'french', 'fusion', 'genoa', 'genovese', 'germain', 'giada', 'gold', 'granule', 'greek', 'hamburger', 'helper', 'herbe', 'hines', 'hodgson', 'hunt', 'instruction', 'interval', 'italianstyle', 'jim', 'jimmy', 'kellogg', 'lagrille', 'lake', 'land',

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    3.1s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    3.2s remaining:   34.7s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    3.2s remaining:   22.3s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    3.2s remaining:   16.1s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    3.3s remaining:   12.5s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    3.3s remaining:    9.9s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    3.3s remaining:    8.1s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    3.4s remaining:    6.7s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    3.4s remaining:    5.6s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    3.4s remaining:    4.8s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    3.4s remaining:    4.1s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.8s remaining:    9.3s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    0.8s remaining:    5.9s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    0.9s remaining:    4.3s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    0.9s remaining:    3.2s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.9s remaining:    2.6s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    0.9s remaining:    2.1s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    0.9s remaining:    1.7s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    0.9s remaining:    1.5s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    0.9s remaining:    1.2s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.9s remaining:    1.1s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.338588 -0.047718       1        1  7.642920
16    -0.072573  0.296123       2        1  6.428171
1      0.070003 -0.129779       3        1  6.101934
19     0.058191 -0.061914       4        1  5.842371
18    -0.239921 -0.142880       5        1  5.668398
9      0.080003 -0.089433       6        1  5.230370
6      0.051689  0.084437       7        1  5.170930
8      0.056188 -0.036156       8        1  5.121359
2      0.094180  0.030428       9        1  5.027408
3      0.076838 -0.053333      10        1  5.025957
15    -0.089584  0.207963      11        1  4.857793
0      0.065555 -0.061549      12        1  4.737268
4     -0.168423 -0.037602      13        1  4.643896
14     0.107157  0.016671      14        1  4.555882
5      0.038973 -0.058190      15        1  4.485092
17    -0.017612 -0.110713      16        1  4.346061
10     0.074619  0.033316      17        1  3.950645
12     0.004606  0.152113      18        1  3.888088
11     0.063224 -0.034628      19        1  3.827274
7      0.085475  0.042846      20        1  3.448184, topic_info=         Term          Freq         Total Category  logprob  loglift
60135    salt  17106.000000  17106.000000  Default  30.0000  30.0000
65531   sugar  14046.000000  14046.000000  Default  29.0000  29.0000
25746  butter  11162.000000  11162.000000  Default  28.0000  28.0000
51246   olive  11211.000000  11211.000000  Default  27.0000  27.0000
50761     oil  16750.000000  16750.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
56379    pork    284.048426   1210.397614  Topic20  -4.5461   1.9178
48006    meat    153.621938    383.473487  Topic20  -5.1608   2.4525
61495    seed    210.029103   3598.000117  Topic20  -4.8480   0.5264
48442    milk    165.719178   4283.791356  Topic20  -5.0850   0.1150
37359     fat    148.183130   1438.364469  Topic20  -5.1968   1.0945

[906 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
376       18  0.995507         Angostura
378       18  0.997285  Angostura bitter
1652       7  0.879425              Bosc
1652       9  0.108955              Bosc
1663       7  0.987775         Bosc pear
...      ...       ...               ...
71854      6  0.711854              zest
71854     16  0.287994              zest
71879      6  0.997757        zest lemon
71903     16  0.990515       zest orange
71952     15  0.997651          zucchini

[1185 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 17, 2, 20, 19, 10, 7, 9, 3, 4, 16, 1, 5, 15, 6, 18, 11, 13, 12, 8])

These topic models/word groupings also don't seem to make much sense, so let's throw this into a TF-IDF and see what happens, even though the authors of LDA don't like doing this.

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()

repo_tfidf_stopwords_aug = tfidf.fit_transform(repo_transformed_stopwords_aug)
tfidf_lda_20_stopwords_aug = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
tfidf_lda_20_repo_transformed_aug = tfidf_lda_20_stopwords_aug.fit_transform(repo_tfidf_stopwords_aug)
pyLDAvis.sklearn.prepare(tfidf_lda_20_stopwords_aug, repo_tfidf_stopwords_aug, cv_stopwords_aug)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    1.8s remaining:   19.4s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    1.8s remaining:   12.5s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    1.8s remaining:    9.0s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    1.9s remaining:    7.2s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    1.9s remaining:    5.8s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    2.0s remaining:    4.7s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    2.0s remaining:    3.9s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    2.0s remaining:    3.3s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    2.0s remaining:    2.8s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    2.0s remaining:    2.4s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.8s remaining:    8.6s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    0.8s remaining:    5.5s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    0.8s remaining:    4.0s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    0.8s remaining:    3.1s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.8s remaining:    2.5s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    0.8s remaining:    2.0s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    0.8s remaining:    1.7s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    0.8s remaining:    1.4s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    0.8s remaining:    1.2s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.8s remaining:    1.0s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.309148 -0.080433       1        1  6.691357
18     0.192795 -0.215736       2        1  6.016977
19    -0.013397  0.181664       3        1  5.875976
16     0.076048  0.159965       4        1  5.641692
1     -0.103489 -0.068852       5        1  5.449186
5     -0.050439 -0.092682       6        1  5.418869
15     0.137666  0.188411       7        1  5.416407
2     -0.080039  0.004504       8        1  5.054227
0     -0.046941 -0.022562       9        1  5.045240
9     -0.037855  0.015409      10        1  5.039256
3     -0.097957 -0.062857      11        1  4.985295
8     -0.072544 -0.022669      12        1  4.916284
6     -0.048637  0.144819      13        1  4.906530
4      0.103019 -0.055788      14        1  4.621243
14    -0.088624 -0.009004      15        1  4.565163
12     0.099029  0.109664      16        1  4.544514
17    -0.042107 -0.063299      17        1  4.201128
10    -0.089147 -0.062248      18        1  4.189428
11    -0.087101 -0.040467      19        1  3.899172
7     -0.059429 -0.007838      20        1  3.522058, topic_info=          Term          Freq         Total Category  logprob  loglift
60135     salt  13212.000000  13212.000000  Default  30.0000  30.0000
65531    sugar  12198.000000  12198.000000  Default  29.0000  29.0000
25746   butter  10265.000000  10265.000000  Default  28.0000  28.0000
45723    lemon   6865.000000   6865.000000  Default  27.0000  27.0000
36582      egg   7190.000000   7190.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
24030  bottled     84.451687    235.661705  Topic20  -5.2520   2.3199
56379     pork     90.209557    534.843608  Topic20  -5.1861   1.5663
50029   nectar     65.890798    119.212864  Topic20  -5.5002   2.7532
48442     milk     71.948135   3181.164329  Topic20  -5.4122  -0.4429
51594    onion     72.515782   6707.417965  Topic20  -5.4044  -1.1810

[947 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
376       16  0.987341         Angostura
378       16  0.988801  Angostura bitter
550        7  0.724088          Armagnac
550       17  0.144818          Armagnac
550       18  0.096545          Armagnac
...      ...       ...               ...
71854     10  0.714838              zest
71854     17  0.284251              zest
71879     13  0.992111        zest lemon
71903     17  0.993166       zest orange
71952      6  0.998717          zucchini

[1674 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 19, 20, 17, 2, 6, 16, 3, 1, 10, 4, 9, 7, 5, 15, 13, 18, 11, 12, 8])

pyLDAvis calls to a deprecated function inside CountVectorizer, which is incompatible with TFIDF. Can we can find an alternate version?